In [30]:
%matplotlib inline
import matplotlib.pyplot as plt

from owslib.etree import etree
from owslib.fes import PropertyIsEqualTo
from owslib.wfs import WebFeatureService

from pydov.search.boring import BoringSearch
from pydov.search.grondwaterfilter import GrondwaterFilterSearch

from pydov.util.location import (
    GmlFilter,
    Within,
)

# Download data from a certain community using its geographic borders

In [40]:
gemeentegrenzen = WebFeatureService(
    'https://geoservices.informatievlaanderen.be/overdrachtdiensten/VRBG/wfs',
    version='1.1.0')

In [41]:
gemeentegrenzen.get_schema('VRBG:Refgem')['properties']

{'DATPUBLBS': 'date',
 'NAAM': 'string',
 'NISCODE': 'string',
 'NUMAC': 'string',
 'OIDN': 'decimal',
 'SHAPE': 'MultiSurfacePropertyType',
 'TERRID': 'decimal',
 'UIDN': 'decimal'}

In [42]:
naam_filter = PropertyIsEqualTo(propertyname='NAAM', literal='Lievegem')

In [43]:
gemeente_poly = gemeentegrenzen.getfeature(
    typename='VRBG:Refgem',
    filter=etree.tostring(naam_filter.toXML()).decode("utf8")).read()

In [46]:
bs = BoringSearch()
df = bs.search(
    location=GmlFilter(gemeente_poly, Within),
    return_fields=('pkey_boring', 'gemeente'))

In [47]:
df.groupby('gemeente').size().reset_index(name='counts')

,gemeente,counts
0,Gent,2
1,Knesselare,2
2,Lovendegem,106
3,Nevele,1
4,Veurne,1
5,Waarschoot,48
6,Zomergem,125


# Download data van bekken

In [99]:
# Initialise WFS service
bekkens = WebFeatureService(
    'https://geoservices.informatievlaanderen.be/overdrachtdiensten/VHAZones/wfs',
    version='1.1.0'
)

In [73]:
# List available types (layers)
list(bekkens.contents)

['VHAZones:Bekken', 'VHAZones:Vhazone', 'VHAZones:Deelbekken']

In [75]:
# Get available fields (properties) of a layer
bekkens.get_schema('VHAZones:Bekken')['properties']

{'BEKNAAM': 'string',
 'BEKNR': 'short',
 'OIDN': 'decimal',
 'SHAPE': 'MultiSurfacePropertyType',
 'STRMGEB': 'string',
 'UIDN': 'decimal'}

In [76]:
# Get distinct values of a field
tree = etree.fromstring(bekkens.getfeature('VHAZones:Bekken', propertyname='BEKNAAM').read())
set((i.text for i in tree.findall('.//{%s}BEKNAAM' % tree.nsmap['VHAZones'])))

{'Bekken Brugse polders',
 'Bekken Gentse kanalen',
 'Beneden-Scheldebekken',
 'Boven-Scheldebekken',
 'Demerbekken',
 'Denderbekken',
 'Dijlebekken',
 'Ijzerbekken',
 'Leiebekken',
 'Maasbekken',
 'Netebekken'}

In [106]:
# Build filter
naam_filter = PropertyIsEqualTo(propertyname='BEKNAAM', literal='Bekken Brugse polders')

In [107]:
# Get features matching filter
bekken_poly = bekkens.getfeature(
    typename='VHAZones:Bekken',
    filter=etree.tostring(naam_filter.toXML()).decode("utf8")).read()

In [108]:
# Get DOV data based on feature geometry
filter_search = GrondwaterFilterSearch()
df = filter_search.search(
    location=GmlFilter(bekken_poly, Within),
    return_fields=('pkey_filter', 'x', 'y')
)

In [109]:
# import the necessary modules (not included in the requirements of pydov!)
import folium
from folium.plugins import MarkerCluster
from pyproj import Proj, transform

In [110]:
# convert the coordinates to lat/lon for folium
def convert_latlon(x1, y1):
    inProj = Proj(init='epsg:31370')
    outProj = Proj(init='epsg:4326')
    x2,y2 = transform(inProj, outProj, x1, y1)
    return x2, y2
df['lon'], df['lat'] = zip(*map(convert_latlon, df['x'], df['y']))
# convert to list
loclist = df[['lat', 'lon']].values.tolist()

In [111]:
# initialize the Folium map on the centre of the selected locations, play with the zoom until ok
fmap = folium.Map(location=[df['lat'].mean(), df['lon'].mean()], zoom_start=10)
marker_cluster = MarkerCluster().add_to(fmap)
for loc in range(0, len(loclist)):
    folium.Marker(loclist[loc], popup=df['pkey_filter'][loc]).add_to(marker_cluster)
fmap